In [2]:
%matplotlib notebook

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib

import scipy as sp
from scipy import sparse
import sklearn

import time
import pickle

from sklearn.metrics import accuracy_score

from sklearn.neighbors import KNeighborsClassifier
from sklearn.decomposition import PCA

import memory_profiler
%load_ext memory_profiler

from pathlib import Path

In [3]:
print(np.__version__)
print(sklearn.__version__)

1.23.1
1.1.1


In [4]:
import black
import jupyter_black

jupyter_black.load(line_length=79)

In [3]:
%load_ext autoreload
%autoreload 2

from pubmed_landscape_src.metrics import knn_accuracy_whitening_scores

In [5]:
variables_path = Path("../results/variables")
figures_path = Path("../results/figures")
berenslab_data_path = Path("/gpfs01/berens/data/data/pubmed_processed")

# Import

In [6]:
%%time
%memit
embeddings_reparsed_all=np.load(berenslab_data_path / 'embeddings/embeddings_reparsed_all.npy')

peak memory: 201.06 MiB, increment: -67.34 MiB


tcmalloc: large alloc 127101853696 bytes == 0x47ba000 @ 


CPU times: user 133 ms, sys: 1min 1s, total: 1min 2s
Wall time: 1min 43s


In [7]:
#save results
colors_new = np.load(variables_path / "colors_new.npy", allow_pickle = True)

In [8]:
pickle_in = open(variables_path / "colors_new_legend.pkl","rb")
colors_new_legend = pickle.load(pickle_in)

## Labels

In [9]:
# define vocabulary swap
new_colors_legend_swap = {value:key for key, value in colors_new_legend.items()}

In [10]:
labels = np.vectorize(new_colors_legend_swap.get)(colors_new)

In [11]:
labels = np.where(colors_new == 'lightgrey', 'unlabeled', labels)

In [12]:
labels[500:550]

array(['unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'cardiology', 'cardiology', 'cardiology',
       'cardiology', 'cardiology', 'cardiology', 'cardiology',
       'cardiology', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled', 'unlabeled',
       'unlabeled', 'unlabeled'], dtype=object)

In [13]:
embeddings_labeled = embeddings_reparsed_all[labels != 'unlabeled']
embeddings_labeled.shape

tcmalloc: large alloc 43768053760 bytes == 0x1dc74d4000 @ 


(7123706, 768)

In [14]:
labels_labeled = labels[labels != 'unlabeled']

## Subset of BERT embeddings

In [79]:
np.random.seed(42)
subset = np.random.choice(embeddings_labeled.shape[0], size=100000, replace=False)
embeddings_subset = embeddings_labeled[subset]
labels_subset = labels_labeled[subset]
embeddings_subset.shape

(100000, 768)

In [80]:
np.save(berenslab_data_path / 'embeddings_subset', embeddings_subset)
np.save(berenslab_data_path / 'labels_subset', labels_subset)

# Experiment

In [ ]:
%%time
knn_accuracy_bert_centered_and_whitened = knn_accuracy_whitening_scores(embeddings_labeled, labels_labeled, ntest=500, rs=23)

tcmalloc: large alloc 43768053760 bytes == 0x322f440000 @ 
tcmalloc: large alloc 43768053760 bytes == 0x3da9248000 @ 
tcmalloc: large alloc 43768053760 bytes == 0x5c42fb0000 @ 
tcmalloc: large alloc 43768053760 bytes == 0x6676428000 @ 
tcmalloc: large alloc 43768053760 bytes == 0x5c42fb0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x3da9248000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 @ 
tcmalloc: large alloc 43764981760 bytes == 0x47dc6c0000 

In [32]:
#rs=23
print(knn_accuracy_bert_centered_and_whitened)

[[0.678 0.678]
 [0.678 0.674]
 [0.642 0.654]]


In [ ]:
np.save(variables_path / "knn_accuracy_bert_centered_and_whitened", knn_accuracy_bert_centered_and_whitened)